In [6]:
import requests
from bs4 import BeautifulSoup
import os
from typing import List, Dict

class MediumScraper:
    def __init__(self, username: str, password: str):
        """
        Initialize the Medium scraper with login credentials
        
        Args:
            username (str): Medium account email or username
            password (str): Medium account password
        """
        self.session = requests.Session()
        self.base_url = 'https://medium.com'
        self.login_url = f'{self.base_url}/login'
        self.username = username
        self.password = password
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
    
    def login(self) -> bool:
        """
        Log in to Medium account
        
        Returns:
            bool: True if login successful, False otherwise
        """
        try:
            # Get login page to retrieve any necessary tokens
            login_page = self.session.get(self.login_url, headers=self.headers)
            
            # Prepare login payload (you might need to adjust based on Medium's current login mechanism)
            login_data = {
                'loginIdentifier': self.username,
                'password': self.password
            }
            
            # Perform login
            response = self.session.post(self.login_url, data=login_data, headers=self.headers)
            
            # Check login success
            return response.status_code == 200
        except Exception as e:
            print(f"Login error: {e}")
            return False
    
    def get_stories(self, profile_url: str) -> List[Dict[str, str]]:
        """
        Fetch stories from a Medium profile
        
        Args:
            profile_url (str): URL of the Medium profile
        
        Returns:
            List[Dict[str, str]]: List of story details
        """
        if not self.login():
            print("Login failed. Cannot fetch stories.")
            return []
        
        try:
            # Fetch profile page
            response = self.session.get(profile_url, headers=self.headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Extract story details
            stories = []
            story_elements = soup.find_all('article')
            
            for story in story_elements:
                title_elem = story.find(['h2', 'h3'])
                link_elem = story.find('a', href=True)
                
                if title_elem and link_elem:
                    story_details = {
                        'title': title_elem.get_text(strip=True),
                        'url': link_elem['href'],
                        'is_member_only': 'member-only' in link_elem.get('class', [])
                    }
                    stories.append(story_details)
            
            return stories
        
        except Exception as e:
            print(f"Error fetching stories: {e}")
            return []
    
    def read_story_content(self, story_url: str) -> str:
        """
        Read the content of a specific story
        
        Args:
            story_url (str): URL of the story
        
        Returns:
            str: Story content or error message
        """
        try:
            response = self.session.get(story_url, headers=self.headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Extract story content (adjust selectors as needed)
            content_elem = soup.find('article')
            return content_elem.get_text(separator='\n', strip=True) if content_elem else "Content not found"
        
        except Exception as e:
            return f"Error reading story: {e}"

def main():
    # Replace with your actual Medium credentials and profile URL
    USERNAME = 'anusonawane666@gmail.com'
    PASSWORD = 'your_password'
    PROFILE_URL = 'https://medium.com/@anushka.sonawane'
    
    # Initialize scraper
    scraper = MediumScraper(USERNAME, PASSWORD)
    
    # Get stories
    stories = scraper.get_stories(PROFILE_URL)
    
    # Print and optionally save stories
    for story in stories:
        print(f"Title: {story['title']}")
        print(f"URL: {story['url']}")
        print(f"Member Only: {story['is_member_only']}")
        
        # Optionally read and save story content
        if story['is_member_only']:
            content = scraper.read_story_content(story['url'])
            with open(f"{story['title']}.txt", 'w', encoding='utf-8') as f:
                f.write(content)
        print('-' * 50)

if __name__ == '__main__':
    main()

Login failed. Cannot fetch stories.
